In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'w') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [3]:
#Prompt
MAIN_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow of a use case.
Then you predict all scenarios that can happen in this flow.

Rules to predict scenarios:
- Stay close to the details described in the flow.
- Choose important cases to generate, important scenario is the scenario that users are more likely to encounter it.
- Limit the appearance of scenarios that are hard to happen. 
- A scenario encompasses a whole function, not just verifying individual steps.
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

SUB_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow and one alternative or exception flow of a use case.
Then you predict all scenarios that can lead user from the main flow to change to the alternative or exception flow mentioned for creating test cases.

Rules to predict scenarios:
- A scenario encompasses a whole function, not just verifying individual steps.
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
- Do not generate scenarios with user analysis. (Example: User accidentally do A and user intentionally do A is the same scenario, so do not consider about "accidentally" or "intentionally" in scenario)
- Do not choose another option that is not chosen by the flow, eventhough it is mentioned (Example: A pop up with OK and Cancel, the flow only has step choose OK. Do not generate scenario that press Cancel)
- Do not generate scenario to test only the main flow.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

EXTRACT_CONDITION_SYSTEM_PROMPT="""
Given use case flows of a feature.
Your task is to identify all the interactive elements within the feature. 
For each interactive element:
Identify what type of that element (button,buttons, icon,scroll, text field,text area, tab, radio buttons, menu, combobox, sliders, switches, dialog, link, form,rating, filter).
Identify all the conditions mentioned in the use case of that element that would make the element valid and the conditions that would make the element invalid based on the description of the use case flow.
Do not arbitrarily create additional conditions that not mention in the use case flow.
Return the element extracted in json format.
The JSON format should follow the following structure:
{"Name of interactive element": {"condition": {valid:"conditions that make element valid", invalid: "conditions that make element invalid"}, "type": "element type"}}
Examples of output json format template: 
{"Username": {"condition": {"valid": "must be over 8 characters and below 30 characters, must be entered", racter, empty"}, "type": "text field"}}
{"Search button": {"condition": {"valid": "", "invalid": ""}, "type": "text field"}}
"""

GEN_SCENARIO_FOR_CONDITION_PROMPT="""
Given a list of interaction element for input value, their extracted conditions and use c"invalid": "below 8 characters, over 30 chaase specification.
For each given invalid condition:
- Generate a test scenario that test that condition.
Do not generate test scenario to test element/condition that not mention in the given element list.
Do not generate test scenario to test valid conditions.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""


FILTER_SYSTEM_PROMPT="""
Your task is to read all of this scenarios generation from many sources.
Then you remove all the same copies because there are many duplicate scenarios in meanings inside the input.
Make sure every scenarios in the response is unique.
Return scenario name only.
"""

# Identify what type of that element (button,buttons, icon,scroll, text field, radio buttons, menu, menu for navigation,menu for opening dialog or another menu, menu for filter, sliders, switches, dialog, link, form,rating, filter).


In [4]:
usecase_name = "Login"
project_name = "MatchaEnglishWebsite"
main_flow_prompt = """
Main flow
Step 1: Learner fills in the username field by a valid username that has been registered
Step 2: Learner fills in the password field by the correct password for the corresponding username
Step 3: Learner press "Login" button
Step 4: System redirects learner to Home page
"""
prompt_all = """
Main flow
Step 1: Learner fills in the username field by a valid username that has been registered
Step 2: Learner fills in the password field by the correct password for the corresponding username
Step 3: Learner press "Login" button
Step 4: System redirects learner to Home page

Alternative flow 1: Login by email
At step 1 of the basic flow: Learner fills in the username field by a valid email that has been registered
Step 2: Learner fills in the password field by the correct password for the corresponding email
Go back to step 3 in the basic flow and continue with the steps from step 3.

Exception flow 1: Wrong password
At step 2 of the basic flow: Learner fills in the password field by a wrong password
Go back to step 3 in the basic flow and continue with the steps from step 3.
At step 4 of the basic flow: System shows a notification that the password is wrong

Exception flow 2: Unregistered username
At step 1 of the basic flow: Learner fills in the username field by a valid username that has not been registered
Go back to step 2 in the basic flow and continue with the steps from step 2.
At step 4 of the basic flow: System shows a notification that this username is not registered yet

Exception flow 3: Blank username fields
At step 1 of the basic flow: Learner does not fill in the username field
Go back to step 2 in the basic flow and continue with the steps from step 2.
At step 4 of the basic flow: System shows a notification that the username field needs to be filled.

Exception flow 4: Blank password fields
At step 2 of the basic flow: Learner does not fill in the password field
Go back to step 3 in the basic flow and take only step 3.
At step 4 of the basic flow: System shows a notification that the password field needs to be filled.
"""
alt_prompt=[
"""Alternative flow 1: Login by email
At step 1 of the basic flow: Learner fills in the username field by a valid email that has been registered
Step 2: Learner fills in the password field by the correct password for the corresponding email
Go back to step 3 in the basic flow and continue with the steps from step 3.
"""]
exc_prompt = ["""Exception flow 1: Wrong password
At step 2 of the basic flow: Learner fills in the password field by a wrong password
Go back to step 3 in the basic flow and continue with the steps from step 3.
At step 4 of the basic flow: System shows a notification that the password is wrong""",
"""Exception flow 2: Unregistered username
At step 1 of the basic flow: Learner fills in the username field by a valid username that has not been registered
Go back to step 2 in the basic flow and continue with the steps from step 2.
At step 4 of the basic flow: System shows a notification that this username is not registered yet""",
"""Exception flow 3: Blank username fields
At step 1 of the basic flow: Learner does not fill in the username field
Go back to step 2 in the basic flow and continue with the steps from step 2.
At step 4 of the basic flow: System shows a notification that the username field needs to be filled.""",
"""Exception flow 4: Blank password fields
At step 2 of the basic flow: Learner does not fill in the password field
Go back to step 3 in the basic flow and take only step 3.
At step 4 of the basic flow: System shows a notification that the password field needs to be filled."""]

In [5]:
promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)
filtercontent = main_gpt_response
print(filtercontent)
for alt in alt_prompt:

  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n\n"+alt}
    ]
  print(main_flow_prompt +"\n\n"+alt)
  sub_gpt_response = ask(promptSubScenario, client, model)
  filtercontent+=  "\n\n" + sub_gpt_response
  print(filtercontent)


for exc in exc_prompt:

  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n\n"+exc}
    ]
  print(main_flow_prompt +"\n\n"+exc)
  sub_gpt_response = ask(promptSubScenario, client, model)
  filtercontent+=  "\n\n" + sub_gpt_response
  print(filtercontent)



promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '\n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)
  filtercontent += "\n" + condition_scenario_response

print(filtercontent)

promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print("filter_gpt_response \n" +filter_gpt_response)
write_string_to_file(f"D:\\GPT-testing\\ResultSet\\5.15\\{project_name}\\{usecase_name}-1.txt", filter_gpt_response)

Based on the provided main flow, here are the scenarios that can happen:

1. Successful Login
2. Incorrect Password
3. Unregistered Username
4. Account Locked After Multiple Failed Attempts

Main flow
Step 1: Learner fills in the username field by a valid username that has been registered
Step 2: Learner fills in the password field by the correct password for the corresponding username
Step 3: Learner press "Login" button
Step 4: System redirects learner to Home page


Alternative flow 1: Login by email
At step 1 of the basic flow: Learner fills in the username field by a valid email that has been registered
Step 2: Learner fills in the password field by the correct password for the corresponding email
Go back to step 3 in the basic flow and continue with the steps from step 3.

Based on the provided main flow, here are the scenarios that can happen:

1. Successful Login
2. Incorrect Password
3. Unregistered Username
4. Account Locked After Multiple Failed Attempts

1. Login with Regis

In [6]:
promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)
filtercontent = main_gpt_response
print(filtercontent)
for alt in alt_prompt:

  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n\n"+alt}
    ]
  print(main_flow_prompt +"\n\n"+alt)
  sub_gpt_response = ask(promptSubScenario, client, model)
  filtercontent+=  "\n\n" + sub_gpt_response
  print(filtercontent)


for exc in exc_prompt:

  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n\n"+exc}
    ]
  print(main_flow_prompt +"\n\n"+exc)
  sub_gpt_response = ask(promptSubScenario, client, model)
  filtercontent+=  "\n\n" + sub_gpt_response
  print(filtercontent)



promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '\n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)
  filtercontent += "\n" + condition_scenario_response

print(filtercontent)

promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print("filter_gpt_response \n" +filter_gpt_response)
write_string_to_file(f"D:\\GPT-testing\\ResultSet\\5.15\\{project_name}\\{usecase_name}-2.txt", filter_gpt_response)

Based on the provided main flow, here are the scenarios that can happen:

1. Successful Login
2. Incorrect Password
3. Username Not Registered
4. Account Locked After Multiple Failed Login Attempts

Main flow
Step 1: Learner fills in the username field by a valid username that has been registered
Step 2: Learner fills in the password field by the correct password for the corresponding username
Step 3: Learner press "Login" button
Step 4: System redirects learner to Home page


Alternative flow 1: Login by email
At step 1 of the basic flow: Learner fills in the username field by a valid email that has been registered
Step 2: Learner fills in the password field by the correct password for the corresponding email
Go back to step 3 in the basic flow and continue with the steps from step 3.

Based on the provided main flow, here are the scenarios that can happen:

1. Successful Login
2. Incorrect Password
3. Username Not Registered
4. Account Locked After Multiple Failed Login Attempts

1. 

In [7]:
promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)
filtercontent = main_gpt_response
print(filtercontent)
for alt in alt_prompt:

  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n\n"+alt}
    ]
  print(main_flow_prompt +"\n\n"+alt)
  sub_gpt_response = ask(promptSubScenario, client, model)
  filtercontent+=  "\n\n" + sub_gpt_response
  print(filtercontent)


for exc in exc_prompt:

  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n\n"+exc}
    ]
  print(main_flow_prompt +"\n\n"+exc)
  sub_gpt_response = ask(promptSubScenario, client, model)
  filtercontent+=  "\n\n" + sub_gpt_response
  print(filtercontent)



promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '\n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)
  filtercontent += "\n" + condition_scenario_response

print(filtercontent)

promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print("filter_gpt_response \n" +filter_gpt_response)
write_string_to_file(f"D:\\GPT-testing\\ResultSet\\5.15\\{project_name}\\{usecase_name}-3.txt", filter_gpt_response)

Based on the provided main flow, here are the scenarios that can happen:

1. Successful Login
2. Incorrect Password
3. Username Not Registered
4. Account Locked After Multiple Failed Login Attempts

Main flow
Step 1: Learner fills in the username field by a valid username that has been registered
Step 2: Learner fills in the password field by the correct password for the corresponding username
Step 3: Learner press "Login" button
Step 4: System redirects learner to Home page


Alternative flow 1: Login by email
At step 1 of the basic flow: Learner fills in the username field by a valid email that has been registered
Step 2: Learner fills in the password field by the correct password for the corresponding email
Go back to step 3 in the basic flow and continue with the steps from step 3.

Based on the provided main flow, here are the scenarios that can happen:

1. Successful Login
2. Incorrect Password
3. Username Not Registered
4. Account Locked After Multiple Failed Login Attempts

1. 

In [8]:
promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)
filtercontent = main_gpt_response
print(filtercontent)
for alt in alt_prompt:

  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n\n"+alt}
    ]
  print(main_flow_prompt +"\n\n"+alt)
  sub_gpt_response = ask(promptSubScenario, client, model)
  filtercontent+=  "\n\n" + sub_gpt_response
  print(filtercontent)


for exc in exc_prompt:

  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n\n"+exc}
    ]
  print(main_flow_prompt +"\n\n"+exc)
  sub_gpt_response = ask(promptSubScenario, client, model)
  filtercontent+=  "\n\n" + sub_gpt_response
  print(filtercontent)



promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '\n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)
  filtercontent += "\n" + condition_scenario_response

print(filtercontent)

promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print("filter_gpt_response \n" +filter_gpt_response)
write_string_to_file(f"D:\\GPT-testing\\ResultSet\\5.15\\{project_name}\\{usecase_name}-4.txt", filter_gpt_response)

Based on the provided main flow, here are the scenarios that can happen:

1. Successful Login
2. Incorrect Password
3. Username Not Registered
4. Account Locked After Multiple Failed Login Attempts

Main flow
Step 1: Learner fills in the username field by a valid username that has been registered
Step 2: Learner fills in the password field by the correct password for the corresponding username
Step 3: Learner press "Login" button
Step 4: System redirects learner to Home page


Alternative flow 1: Login by email
At step 1 of the basic flow: Learner fills in the username field by a valid email that has been registered
Step 2: Learner fills in the password field by the correct password for the corresponding email
Go back to step 3 in the basic flow and continue with the steps from step 3.

Based on the provided main flow, here are the scenarios that can happen:

1. Successful Login
2. Incorrect Password
3. Username Not Registered
4. Account Locked After Multiple Failed Login Attempts

1. 

In [9]:
promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)
filtercontent = main_gpt_response
print(filtercontent)
for alt in alt_prompt:

  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n\n"+alt}
    ]
  print(main_flow_prompt +"\n\n"+alt)
  sub_gpt_response = ask(promptSubScenario, client, model)
  filtercontent+=  "\n\n" + sub_gpt_response
  print(filtercontent)


for exc in exc_prompt:

  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n\n"+exc}
    ]
  print(main_flow_prompt +"\n\n"+exc)
  sub_gpt_response = ask(promptSubScenario, client, model)
  filtercontent+=  "\n\n" + sub_gpt_response
  print(filtercontent)



promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '\n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)
  filtercontent += "\n" + condition_scenario_response

print(filtercontent)

promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print("filter_gpt_response \n" +filter_gpt_response)
write_string_to_file(f"D:\\GPT-testing\\ResultSet\\5.15\\{project_name}\\{usecase_name}-5.txt", filter_gpt_response)

Based on the provided main flow, here are the scenarios that can be predicted:

1. Successful Login
2. Incorrect Password
3. Username Not Registered
4. Account Locked After Multiple Failed Login Attempts

Main flow
Step 1: Learner fills in the username field by a valid username that has been registered
Step 2: Learner fills in the password field by the correct password for the corresponding username
Step 3: Learner press "Login" button
Step 4: System redirects learner to Home page


Alternative flow 1: Login by email
At step 1 of the basic flow: Learner fills in the username field by a valid email that has been registered
Step 2: Learner fills in the password field by the correct password for the corresponding email
Go back to step 3 in the basic flow and continue with the steps from step 3.

Based on the provided main flow, here are the scenarios that can be predicted:

1. Successful Login
2. Incorrect Password
3. Username Not Registered
4. Account Locked After Multiple Failed Login A